
# IMPORTAÇÕES

In [1]:
import os
import glob
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from datetime import date
from datetime import datetime
import calendar
from openpyxl.utils import get_column_letter
from datetime import datetime, timedelta

# AJUSTE DA BASE BRUTA

In [2]:
# Diretório de downloads
diretorio_downloads = os.path.join(os.path.expanduser('~'), 'Downloads')

# Padrão de nome do arquivo
padrao_nome_arquivo = 'resumo_semanal_lpc*'

# Caminho completo para o arquivo de saída na pasta de downloads
arquivo_saida = os.path.join(diretorio_downloads, 'resumo_semanal_modificado.xlsx')

# Listar todos os arquivos que correspondem ao padrão de nome
arquivos = glob.glob(os.path.join(diretorio_downloads, padrao_nome_arquivo))

# Verificar se há arquivos correspondentes
if arquivos:
    # Selecionar o último arquivo pela data de modificação
    arquivo_mais_recente = max(arquivos, key=os.path.getmtime)
    
    # Carregar o arquivo Excel
    workbook = pd.ExcelFile(arquivo_mais_recente)
    
    # Criar um escritor Excel
    writer = pd.ExcelWriter(arquivo_saida, engine='xlsxwriter')
    
    # Configurar um objeto de formatação para datas
    fmt = writer.book.add_format({'num_format': 'dd/mm/yyyy'})
    
    # Ler o conteúdo da primeira aba para obter a data
    df_primeira_aba = workbook.parse(workbook.sheet_names[0], header=None)
    cabecalho_index = df_primeira_aba[df_primeira_aba.iloc[:, 0] == 'DATA INICIAL'].index[0]
    data_atual = pd.to_datetime(df_primeira_aba.iloc[cabecalho_index + 1, 0]).strftime('%d/%m/%Y')
    data_anterior = (pd.to_datetime(data_atual, format='%d/%m/%Y') - pd.DateOffset(days=7)).strftime('%d/%m/%Y')
    
    # Imprimir as datas
    print(f'Data atual: {data_atual}')
    print(f'Data anterior: {data_anterior}')
    
    # Iterar sobre as abas (tabelas)
    for nome_aba in workbook.sheet_names:
        # Ler o conteúdo da aba em um DataFrame
        df_temporario = workbook.parse(nome_aba, header=None)  # Não usar cabeçalho
        cabecalho_index = df_temporario[df_temporario.iloc[:, 0] == 'DATA INICIAL'].index[0]
        df_temporario.columns = df_temporario.iloc[cabecalho_index]  # Usar a linha correta como cabeçalho
        df_temporario = df_temporario.iloc[cabecalho_index+1:]  # Excluir linhas anteriores ao cabeçalho
        
        # Formatar as datas no DataFrame
        for coluna in df_temporario.columns:
            if 'DATA' in coluna:
                df_temporario[coluna] = pd.to_datetime(df_temporario[coluna], dayfirst=True)
        
        # Salvar o DataFrame como uma aba no arquivo Excel
        df_temporario.to_excel(writer, sheet_name=nome_aba, index=False, na_rep='')
        
        # Aplicar a formatação de datas na aba
        worksheet = writer.sheets[nome_aba]
        for idx, col in enumerate(df_temporario.columns):
            if 'DATA' in col:
                worksheet.set_column(idx, idx, None, fmt)
    
    # Fechar o escritor Excel
    writer.save()
    
    print(f'Arquivo salvo com sucesso em {arquivo_saida}')
else:
    print("Nenhum arquivo encontrado com o padrão de nome especificado.")

Data atual: 10/03/2024
Data anterior: 03/03/2024
Arquivo salvo com sucesso em C:\Users\ricardo\Downloads\resumo_semanal_modificado.xlsx


# REGISTRANDO ABAS

In [3]:
# Caminho do arquivo Excel
caminho_resumo_modificado = r'C:\Users\ricardo\Downloads\resumo_semanal_modificado.xlsx'

# Carregar o arquivo Excel
workbook_resumo = load_workbook(filename=caminho_resumo_modificado, read_only=True)

In [4]:
# Listar todas as abas
abas = workbook_resumo.sheetnames

# Fechar o arquivo Excel
workbook_resumo.close()

# Mostrar as abas
print("Abas presentes no arquivo:")
for aba in abas:
    print(aba)


Abas presentes no arquivo:
CAPITAIS
MUNICIPIOS
ESTADOS
REGIOES
BRASIL


In [5]:
# Caminho do arquivo Excel
caminho_arquivo = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'

# Carregar o arquivo Excel
workbook_base = load_workbook(filename=caminho_arquivo, read_only=True)

In [6]:
# Listar todas as abas
abas = workbook_base.sheetnames

# Fechar o arquivo Excel
workbook_base.close()

# Mostrar as abas
print("Abas presentes no arquivo:")
for aba in abas:
    print(aba)

Abas presentes no arquivo:
Panorama geral de qtd pesquisas
New Gas Database
Capitais-
Serie Historica def
New Gas Database - SP ANO
PROC REGIOES-ESTADOS
CAPITAIS
MUNICIPIOS
ESTADOS
REGIOES
BRASIL


# ARMAZENAMENTO NA BASE TRATADA

In [7]:
# Caminho da planilha base
caminho_planilha_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'

# Caminho do arquivo resumo_semanal_modificado
caminho_resumo_modificado = r'C:\Users\ricardo\Downloads\resumo_semanal_modificado.xlsx'

# Carregar a planilha base
workbook_base = load_workbook(filename=caminho_planilha_base)

In [8]:
# Lista de abas a serem transferidas
abas = ['CAPITAIS', 'MUNICIPIOS', 'ESTADOS', 'REGIOES', 'BRASIL']

# Carregar o arquivo resumo_semanal_modificado
df_resumo_modificado = pd.read_excel(caminho_resumo_modificado)

# Carregar o arquivo resumo_semanal_modificado
xls = pd.ExcelFile(caminho_resumo_modificado)

# Obter a data atual
data_atual = df_resumo_modificado.iloc[0]['DATA INICIAL']

# Para cada aba a ser transferida
for aba in abas:
    # Verificar se a aba está presente no arquivo modificado
    if aba in xls.sheet_names:
        # Carregar os dados da aba correspondente no arquivo modificado
        df_aba_modificada = pd.read_excel(caminho_resumo_modificado, sheet_name=aba)
        
        # Carregar a aba correspondente na planilha base
        aba_base = workbook_base[aba]
        
        # Verificar se a data atual já está presente na coluna "DATA INICIAL"
        datas_na_aba = [aba_base.cell(row=row, column=1).value for row in range(2, aba_base.max_row + 1)]
        if data_atual in datas_na_aba:
            print(f'Dados já atualizados na aba {aba}')
        else:
            # Encontrar a primeira célula vazia na coluna A
            idx = 1
            while aba_base.cell(row=idx, column=1).value is not None:
                idx += 1
            
            # Preencher os valores na coluna correspondente na planilha base
            for i, row in df_aba_modificada.iterrows():
                for j, value in enumerate(row):
                    aba_base.cell(row=idx, column=j+1, value=value)
                idx += 1

In [9]:
# Salvar as alterações na planilha base como um arquivo XLSX
workbook_base.save(caminho_planilha_base)

print(f'Alterações salvas na planilha base como {caminho_planilha_base}')

Alterações salvas na planilha base como Z:\IPTC\13 - Analista de Dados_Bases\01 - Painel do Diesel - Bases\Ricardo\base_diesel_desde__31_07_2022.xlsx


# PREÇO MÉDIO DE REVENDA DAS CAPITAIS NA ULTIMA SEMANA - OLEO DIESEL E OLEO DIESEL S10

In [10]:
# Caminho do arquivo base
caminho_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'

# Carregar o arquivo base
workbook_base = load_workbook(filename=caminho_base)

In [11]:
# Selecionar a aba "CAPITAIS" no arquivo base
aba_capitais_base = workbook_base['CAPITAIS']

# Converter a aba "CAPITAIS" em um DataFrame
df_capitais_base = pd.DataFrame(aba_capitais_base.values, columns=[cell.value for cell in aba_capitais_base[1]])

# Converter a coluna 'DATA INICIAL' para o formato de data, pulando a primeira linha
df_capitais_base['DATA INICIAL'] = pd.to_datetime(df_capitais_base['DATA INICIAL'], errors='coerce')

# Formatar as datas na coluna 'DATA INICIAL' para o formato 'dd/mm/aaaa'
df_capitais_base['DATA INICIAL'] = df_capitais_base['DATA INICIAL'].dt.strftime('%d/%m/%Y')

# Lista de capitais
capitais = [
    'ARACAJU', 'BELEM', 'BELO HORIZONTE', 'BOA VISTA', 'BRASILIA', 'CAMPO GRANDE',
    'CUIABA', 'CURITIBA', 'FLORIANOPOLIS', 'FORTALEZA', 'GOIANIA', 'JOAO PESSOA',
    'MACAPA', 'MACEIO', 'MANAUS', 'NATAL', 'PALMAS', 'PORTO ALEGRE', 'PORTO VELHO',
    'RECIFE', 'RIO BRANCO', 'RIO DE JANEIRO', 'SALVADOR', 'SAO LUIS', 'SAO PAULO', 'TERESINA', 'VITORIA'
]

# Formatar a data atual para o formato 'dd/mm/aaaa'
data_atual_formatada = data_atual.strftime('%d/%m/%Y')

# Filtrar os dados da aba 'CAPITAIS' para incluir apenas as capitais da lista e a data atual
df_capitais_filtrado_base = df_capitais_base[df_capitais_base['DATA INICIAL'] == data_atual_formatada]
df_capitais_filtrado_base = df_capitais_filtrado_base[df_capitais_filtrado_base['MUNICÍPIO'].isin(capitais)]

# Pivotar a tabela para ter os produtos como colunas
df_pivot_base = df_capitais_filtrado_base.pivot(index='MUNICÍPIO', columns='PRODUTO', values='PREÇO MÉDIO REVENDA')

# Selecionar apenas as linhas de interesse (óleo diesel comum e óleo diesel S10)
df_resultado_base = df_pivot_base[['OLEO DIESEL', 'OLEO DIESEL S10']]

# Exibir o resultado
print(df_resultado_base)


PRODUTO        OLEO DIESEL OLEO DIESEL S10
MUNICÍPIO                                 
ARACAJU               6.24            5.93
BELEM                  NaN            5.58
BELO HORIZONTE        5.66            5.83
BOA VISTA             6.63            6.68
BRASILIA              5.83            5.92
CAMPO GRANDE          5.89            5.96
CUIABA                 5.9            5.91
CURITIBA              5.62             5.9
FLORIANOPOLIS          NaN            6.06
FORTALEZA              NaN            5.79
GOIANIA               5.82            5.94
JOAO PESSOA           5.64            5.74
MACAPA                6.59            6.54
MACEIO                 NaN            5.61
MANAUS                6.29            6.29
NATAL                 5.96            5.97
PALMAS                5.69            5.64
PORTO ALEGRE          5.86            5.89
PORTO VELHO           6.44             6.4
RECIFE                 NaN            5.67
RIO BRANCO             7.1            7.15
RIO DE JANE

In [12]:
# Remover as linhas com valores NaN
df_resultado_limpo = df_resultado_base.dropna()

# Resetar o índice para transformar "MUNICÍPIO" em uma coluna
df_resultado_limpo.reset_index(inplace=True)

# Renomear a coluna "MUNICÍPIO" para "CAPITAIS"
df_resultado_limpo.rename(columns={'MUNICÍPIO': 'CAPITAIS'}, inplace=True)

# Selecionar apenas as colunas desejadas
df_resultado_limpo = df_resultado_limpo[['CAPITAIS', 'OLEO DIESEL S10', 'OLEO DIESEL']]

# Exibir o DataFrame resultante
print(df_resultado_limpo)


PRODUTO        CAPITAIS OLEO DIESEL S10 OLEO DIESEL
0               ARACAJU            5.93        6.24
1        BELO HORIZONTE            5.83        5.66
2             BOA VISTA            6.68        6.63
3              BRASILIA            5.92        5.83
4          CAMPO GRANDE            5.96        5.89
5                CUIABA            5.91         5.9
6              CURITIBA             5.9        5.62
7               GOIANIA            5.94        5.82
8           JOAO PESSOA            5.74        5.64
9                MACAPA            6.54        6.59
10               MANAUS            6.29        6.29
11                NATAL            5.97        5.96
12               PALMAS            5.64        5.69
13         PORTO ALEGRE            5.89        5.86
14          PORTO VELHO             6.4        6.44
15           RIO BRANCO            7.15         7.1
16       RIO DE JANEIRO            6.05        5.89
17             SALVADOR               6        5.66
18          

C:\Users\ricardo\AppData\Local\Temp\ipykernel_13608\2768604988.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado_limpo.rename(columns={'MUNICÍPIO': 'CAPITAIS'}, inplace=True)


In [13]:
# Carregar a planilha base
caminho_planilha_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'
workbook_base = load_workbook(filename=caminho_planilha_base)

In [14]:
# Selecionar a aba "Capitais-" na planilha base
worksheet = workbook_base['Capitais-']

# Limpar a aba "Capitais-" antes de colar os novos dados
worksheet.delete_rows(1, worksheet.max_row)


# Colar o DataFrame na aba "Capitais-"
for r_idx, row in enumerate(dataframe_to_rows(df_resultado_limpo, index=True, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = worksheet.cell(row=r_idx, column=c_idx, value=value)
        cell.alignment = Alignment(horizontal='center', vertical='center')


# Adicionar a data atual na última célula da primeira linha
worksheet.cell(row=1, column=worksheet.max_column + 1, value=date.today().strftime('%d/%m/%Y'))
worksheet.cell(row=1, column=worksheet.max_column).alignment = Alignment(horizontal='center', vertical='center')

# Excluir a coluna A da planilha
worksheet.delete_cols(1)
# Excluir a linha 2 da planilha
worksheet.delete_rows(2)

In [15]:
# Salvar as alterações na planilha base
workbook_base.save(caminho_planilha_base)
print(f'Dados colados com sucesso na aba "Capitais-" da planilha base.')

Dados colados com sucesso na aba "Capitais-" da planilha base.


# ATUALIZAÇÃO MUNICÍPIOS ÚLTIMA SEMANA - OLEO DIESEL E OLEO DIESEL S10

In [16]:
# Carregar a planilha base
caminho_planilha_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'
workbook_base = load_workbook(filename=caminho_planilha_base)

In [17]:
# Selecionar a aba "MUNICIPIOS" no arquivo base
aba_municipios_base = workbook_base['MUNICIPIOS']

# Converter a aba "MUNICIPIOS" em um DataFrame
df_municipios_base = pd.DataFrame(aba_municipios_base.values, columns=[cell.value for cell in aba_municipios_base[1]])

# Converter a coluna 'DATA INICIAL' para o formato de data, pulando a primeira linha
df_municipios_base['DATA INICIAL'] = pd.to_datetime(df_municipios_base['DATA INICIAL'], errors='coerce')
df_municipios_base['DATA FINAL'] = pd.to_datetime(df_municipios_base['DATA FINAL'], errors='coerce')

# Formatar as datas na coluna 'DATA INICIAL' e 'DATA FINAL' para o formato 'dd/mm/aaaa'
df_municipios_base['DATA INICIAL'] = df_municipios_base['DATA INICIAL'].dt.strftime('%d/%m/%Y')
df_municipios_base['DATA FINAL'] = df_municipios_base['DATA FINAL'].dt.strftime('%d/%m/%Y')

# Formatar a data atual para o formato 'dd/mm/aaaa'
data_atual_formatada = data_atual.strftime('%d/%m/%Y')

# Filtrar os dados da aba 'MUNICIPIOS' para incluir apenas os municipios da lista e a data atual
df_municipios_filtrado_base = df_municipios_base[df_municipios_base['DATA INICIAL'] == data_atual_formatada]
df_municipios_filtrado_diesel = df_municipios_filtrado_base[df_municipios_filtrado_base['PRODUTO'].isin(['OLEO DIESEL', 'OLEO DIESEL S10'])]

df_municipios_filtrado_diesel.head()


,DATA INICIAL,DATA FINAL,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,COEF DE VARIAÇÃO REVENDA
201590,10/03/2024,16/03/2024,PARA,ABAETETUBA,OLEO DIESEL,1,R$/l,6,0,6,6,0
201591,10/03/2024,16/03/2024,MARANHAO,ACAILANDIA,OLEO DIESEL,9,R$/l,5.63,0.093868,5.55,5.79,0.017
201592,10/03/2024,16/03/2024,SAO PAULO,ADAMANTINA,OLEO DIESEL,1,R$/l,5.45,0,5.45,5.45,0
201593,10/03/2024,16/03/2024,GOIAS,AGUAS LINDAS DE GOIAS,OLEO DIESEL,6,R$/l,6.24,0.151658,5.99,6.39,0.024
201594,10/03/2024,16/03/2024,BAHIA,ALAGOINHAS,OLEO DIESEL,7,R$/l,5.44,0.167118,5.29,5.78,0.031


In [18]:
num_linhas = df_municipios_filtrado_diesel.shape[0]
print(num_linhas)

869


In [19]:
# Carregar a planilha base
caminho_planilha_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'
workbook_base = load_workbook(filename=caminho_planilha_base)

In [20]:
# Selecionar a aba "New Gas Database" no arquivo base
aba_new_gas_database = workbook_base['New Gas Database']

# Limpar a aba "New Gas Database" antes de colar os novos dados
aba_new_gas_database.delete_rows(1, aba_new_gas_database.max_row)

# Converter o DataFrame para linhas
rows = dataframe_to_rows(df_municipios_filtrado_diesel, index=False, header=True)

# Colar os dados na aba "New Gas Database"
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
        aba_new_gas_database.cell(row=r_idx, column=c_idx, value=value)
        
# Deletar as colunas especificadas
columns_to_delete = ['NÚMERO DE POSTOS PESQUISADOS', 'UNIDADE DE MEDIDA', 'DESVIO PADRÃO REVENDA', 'COEF DE VARIAÇÃO REVENDA']
for column_name in columns_to_delete:
    column_index = next(idx for idx, cell in enumerate(aba_new_gas_database[1], 1) if cell.value == column_name)
    aba_new_gas_database.delete_cols(column_index)

# Criar a coluna "Ano" na coluna A
ano_atual = date.today().year
aba_new_gas_database.insert_cols(1)
aba_new_gas_database.cell(row=1, column=1, value='Ano')
for row in range(2, aba_new_gas_database.max_row + 1):
    aba_new_gas_database.cell(row=row, column=1, value=ano_atual)
    
# Obter a letra da coluna 'J'
column_letter = get_column_letter(10)  # 'J' é a décima letra do alfabeto
# Adicionar a data atual na célula J1
aba_new_gas_database[column_letter + '1'] = date.today().strftime('%d/%m/%Y')

In [21]:
# Salvar as alterações na planilha base
workbook_base.save(caminho_planilha_base)
print(f'Dados colados com sucesso na aba "New Gas Database" da planilha base.')


Dados colados com sucesso na aba "New Gas Database" da planilha base.


# ATUALIZAR PREÇO MÉDIO OLEO DIESEL E OLEO DIESEL S10 DE SP

In [22]:
# Carregar a planilha base
caminho_planilha_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'
workbook_base = load_workbook(filename=caminho_planilha_base)

In [23]:
# Acessar as abas relevantes
aba_capitais = workbook_base['CAPITAIS']
aba_sp_ano = workbook_base['New Gas Database - SP ANO']

# Extrair o ano e mês da data atual
ano_atual = data_atual.year
mes_atual = data_atual.month

# Converter a aba "CAPITAIS" em um DataFrame
df_capitais_base = pd.DataFrame(aba_capitais.values, columns=[cell.value for cell in aba_capitais_base[1]])

# Converter a coluna 'DATA INICIAL' para o formato de data, pulando a primeira linha
df_capitais_mes_atual = pd.DataFrame(aba_capitais.values)
df_capitais_mes_atual.columns = df_capitais_mes_atual.iloc[0]  # Definir a primeira linha como cabeçalho
df_capitais_mes_atual = df_capitais_mes_atual.iloc[1:]  # Remover a primeira linha (cabeçalho)

# Converter a coluna 'DATA INICIAL' para o tipo datetime
df_capitais_mes_atual['DATA INICIAL'] = pd.to_datetime(df_capitais_mes_atual['DATA INICIAL'], errors='coerce')

# Filtrar os dados na aba "CAPITAIS" pelo ano e mês atual
df_capitais_mes_atual = df_capitais_mes_atual[df_capitais_mes_atual['DATA INICIAL'].dt.year == ano_atual]
df_capitais_mes_atual = df_capitais_mes_atual[df_capitais_mes_atual['DATA INICIAL'].dt.month == mes_atual]
df_capitais_mes_atual = df_capitais_mes_atual[df_capitais_mes_atual['ESTADO'].isin(['SAO PAULO'])]

# Filtrar os dados na aba "CAPITAIS" pelo PRODUTO OLEO DIESEL
df_capitais_mes_atual_diesel = df_capitais_mes_atual[df_capitais_mes_atual['PRODUTO'].isin(['OLEO DIESEL'])]
df_capitais_mes_atual_diesel_s10 = df_capitais_mes_atual[df_capitais_mes_atual['PRODUTO'].isin(['OLEO DIESEL S10'])]

# Calcular a média dos valores da coluna 'PREÇO MÉDIO REVENDA'
media_preco_diesel = df_capitais_mes_atual_diesel['PREÇO MÉDIO REVENDA'].mean()
media_preco_diesel_s10 = df_capitais_mes_atual_diesel_s10['PREÇO MÉDIO REVENDA'].mean()

# Transformar o mês atual para o formato de data
data_mes_atual = datetime(ano_atual, mes_atual, 1)

# Formatar a data para o formato desejado
data_mes_formatada = data_mes_atual.strftime('%d/%m/%Y %H:%M')

In [24]:
# Mapear o número do mês para o nome abreviado em português
meses_abreviados = {1: 'jan', 2: 'fev', 3: 'mar', 4: 'abr', 5: 'mai', 6: 'jun', 7: 'jul', 8: 'ago', 9: 'set', 10: 'out', 11: 'nov', 12: 'dez'}

# Carregar a aba 'New Gas Database - SP ANO' como um DataFrame
df_sp_ano = pd.DataFrame(aba_sp_ano.values)
df_sp_ano.columns = df_sp_ano.iloc[0]  # Definir a primeira linha como cabeçalho
df_sp_ano = df_sp_ano.iloc[1:]  # Remover a primeira linha (cabeçalho)

# Converter o número do mês para o nome abreviado em português
mes_atual_nome = meses_abreviados[mes_atual]

# Obter os dois últimos dígitos do ano atual
ano_atual_dois_digitos = ano_atual % 100

# Verificar se a data atual está presente na coluna 'DATA FINAL' de df_sp_ano
if data_mes_formatada in df_sp_ano['DATA FINAL'].values:
    # Atualizar os valores da coluna 'PREÇO MÉDIO REVENDA' conforme o produto
    df_sp_ano.loc[(df_sp_ano['DATA FINAL'] == data_mes_formatada) & (df_sp_ano['PRODUTO'] == 'OLEO DIESEL'), 'PREÇO MÉDIO REVENDA'] = media_preco_diesel
    df_sp_ano.loc[(df_sp_ano['DATA FINAL'] == data_mes_formatada) & (df_sp_ano['PRODUTO'] == 'OLEO DIESEL S10'), 'PREÇO MÉDIO REVENDA'] = media_preco_diesel_s10
    print('Já contém o mês atual em questão, valores atualizados!')
else:
    # Adicionar uma nova linha com os valores adequados
    nova_linha_diesel = {'DATA FINAL': data_mes_formatada, 'PRODUTO': 'OLEO DIESEL', 'PREÇO MÉDIO REVENDA': media_preco_diesel, 'Mes-Ano': mes_atual_nome.capitalize(), 'Ano': ano_atual_dois_digitos, 'concat': f"{mes_atual_nome.lower()}/.{ano_atual_dois_digitos}"}
    nova_linha_diesel_s10 = {'DATA FINAL': data_mes_formatada, 'PRODUTO': 'OLEO DIESEL S10', 'PREÇO MÉDIO REVENDA': media_preco_diesel_s10, 'Mes-Ano': mes_atual_nome.capitalize(), 'Ano': ano_atual_dois_digitos, 'concat': f"{mes_atual_nome.lower()}/.{ano_atual_dois_digitos}"}
    df_sp_ano = df_sp_ano.append(nova_linha_diesel, ignore_index=True)
    df_sp_ano = df_sp_ano.append(nova_linha_diesel_s10, ignore_index=True)
    print('Não possui o mês em questão, adicionando novas linhas com os valores referente ao mês em questão!')
# Exibir as últimas 6 linhas de df_sp_ano
ultimas_linhas_sp_ano = df_sp_ano.tail(6)
print(ultimas_linhas_sp_ano)

Já contém o mês atual em questão, valores atualizados!
0          DATA FINAL          PRODUTO PREÇO MÉDIO REVENDA Mes-Ano Ano  \
109  01/01/2024 00:00      OLEO DIESEL                5.81     jan  24   
110  01/01/2024 00:00  OLEO DIESEL S10                6.09     jan  24   
111  01/02/2024 00:00      OLEO DIESEL              5.8075     Fev  24   
112  01/02/2024 00:00  OLEO DIESEL S10              6.0775     Fev  24   
113  01/03/2024 00:00      OLEO DIESEL                5.81     Mar  24   
114  01/03/2024 00:00  OLEO DIESEL S10                6.07     Mar  24   

0     concat 11/03/2024  
109  jan./24       None  
110  jan./24       None  
111  fev/.24       None  
112  fev/.24       None  
113  mar/.24       None  
114  mar/.24       None  


In [25]:
# Limpar a aba "New Gas Database - SP ANO" antes de colar os novos dados
aba_sp_ano.delete_rows(1, aba_sp_ano.max_row)

# Converter o DataFrame para linhas
rows = dataframe_to_rows(df_sp_ano, index=False, header=True)

# Colar os dados na aba "New Gas Database - SP ANO"
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
        aba_sp_ano.cell(row=r_idx, column=c_idx, value=value)
        
# Obter a letra da coluna 'G'
column_letter = get_column_letter(7)  # 'G' é a sétima letra do alfabeto
# Adicionar a data atual na célula G1
aba_sp_ano[column_letter + '1'] = date.today().strftime('%d/%m/%Y')

In [26]:
# Salvar as alterações na planilha base
workbook_base.save(caminho_planilha_base)
print(f'Dados colados com sucesso na aba "New Gas Database" da planilha base.')

Dados colados com sucesso na aba "New Gas Database" da planilha base.


# ATUALIZAR PREÇO MÉDIO OLEO DIESEL E OLEO DIESEL S10 DO BRASIL

In [27]:
# Carregar a planilha base
caminho_planilha_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'
workbook_base = load_workbook(filename=caminho_planilha_base)

In [28]:
df_capitais_mes_atual.head()

,DATA INICIAL,DATA FINAL,ESTADO,MUNICÍPIO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,COEF DE VARIAÇÃO REVENDA
13989,2024-03-03,2024-03-09 00:00:00,SAO PAULO,SAO PAULO,ETANOL HIDRATADO,184,R$/l,3.42,0.382345,2.83,4.99,0.112
14016,2024-03-03,2024-03-09 00:00:00,SAO PAULO,SAO PAULO,GASOLINA ADITIVADA,184,R$/l,5.99,0.682905,4.99,7.99,0.114
14043,2024-03-03,2024-03-09 00:00:00,SAO PAULO,SAO PAULO,GASOLINA COMUM,180,R$/l,5.64,0.64307,4.69,7.97,0.114
14070,2024-03-03,2024-03-09 00:00:00,SAO PAULO,SAO PAULO,GLP,54,R$/13kg,105.77,9.40982,84.99,125,0.089
14085,2024-03-03,2024-03-09 00:00:00,SAO PAULO,SAO PAULO,GNV,21,R$/m³,4.17,0.262874,3.79,4.9,0.063


In [29]:
# Acessar as abas relevantes
aba_brasil = workbook_base['BRASIL']
aba_brasil_ano = workbook_base['Serie Historica def']

# Extrair o ano e mês da data atual
ano_atual = data_atual.year
mes_atual = data_atual.month

# Converter a aba "BRASIL" em um DataFrame
df_brasil_base = pd.DataFrame(aba_brasil.values, columns=[cell.value for cell in aba_brasil[1]])

# Converter a coluna 'DATA INICIAL' para o formato de data, pulando a primeira linha
df_brasil_mes_atual = pd.DataFrame(aba_brasil.values)
df_brasil_mes_atual.columns = df_brasil_mes_atual.iloc[0]  # Definir a primeira linha como cabeçalho
df_brasil_mes_atual = df_brasil_mes_atual.iloc[1:]  # Remover a primeira linha (cabeçalho)

# Converter a coluna 'DATA INICIAL' para o tipo datetime
df_brasil_mes_atual['DATA INICIAL'] = pd.to_datetime(df_brasil_mes_atual['DATA INICIAL'], errors='coerce')

# Filtrar os dados na aba "BRASIL" pelo ano e mês atual
df_brasil_mes_atual = df_brasil_mes_atual[df_brasil_mes_atual['DATA INICIAL'].dt.year == ano_atual]
df_brasil_mes_atual = df_brasil_mes_atual[df_brasil_mes_atual['DATA INICIAL'].dt.month == mes_atual]

# Filtrar os dados na aba "CAPITAIS" pelo PRODUTO OLEO DIESEL
df_brasil_mes_atual_diesel = df_brasil_mes_atual[df_brasil_mes_atual['PRODUTO'].isin(['OLEO DIESEL'])]
df_brasil_mes_atual_diesel_s10 = df_brasil_mes_atual[df_brasil_mes_atual['PRODUTO'].isin(['OLEO DIESEL S10'])]

# Calcular a média dos valores da coluna 'PREÇO MÉDIO REVENDA'
media_brasil_preco_diesel = df_brasil_mes_atual_diesel['PREÇO MÉDIO REVENDA'].mean()
media_brasil_preco_diesel_s10 = df_brasil_mes_atual_diesel_s10['PREÇO MÉDIO REVENDA'].mean()

# Transformar o mês atual para o formato de data
data_mes_atual = datetime(ano_atual, mes_atual, 1)

# Formatar a data para o formato desejado
data_mes_formatada = data_mes_atual.strftime('%d/%m/%Y %H:%M')

In [30]:
print(media_brasil_preco_diesel)
print(media_brasil_preco_diesel_s10)

5.87
5.9350000000000005


In [31]:
# Mapear o número do mês para o nome abreviado em português
meses_abreviados = {1: 'jan', 2: 'fev', 3: 'mar', 4: 'abr', 5: 'mai', 6: 'jun', 7: 'jul', 8: 'ago', 9: 'set', 10: 'out', 11: 'nov', 12: 'dez'}

# Carregar a aba 'New Gas Database - SP ANO' como um DataFrame
df_brasil_ano = pd.DataFrame(aba_brasil_ano.values)
df_brasil_ano.columns = df_brasil_ano.iloc[0]  # Definir a primeira linha como cabeçalho
df_brasil_ano = df_brasil_ano.iloc[1:]  # Remover a primeira linha (cabeçalho)

# Converter o número do mês para o nome abreviado em português
mes_atual_nome = meses_abreviados[mes_atual]

# Obter os dois últimos dígitos do ano atual
ano_atual_dois_digitos = ano_atual % 100

# Padronizar a data atual para o formato "mm.-aaaa"
data_mes_formatada = f"{mes_atual_nome}.-{ano_atual_dois_digitos}"

# Verificar se a data atual está presente na coluna 'DATA FINAL' de df_sp_ano
if data_mes_formatada in df_brasil_ano['MÊS'].values:
    # Atualizar os valores da coluna 'PREÇO MÉDIO REVENDA' conforme o produto
    df_brasil_ano.loc[(df_brasil_ano['MÊS'] == data_mes_formatada) & (df_brasil_ano['PRODUTO'] == 'OLEO DIESEL'), 'PREÇO MÉDIO REVENDA'] = media_brasil_preco_diesel
    df_brasil_ano.loc[(df_brasil_ano['MÊS'] == data_mes_formatada) & (df_brasil_ano['PRODUTO'] == 'OLEO DIESEL S10'), 'PREÇO MÉDIO REVENDA'] = media_brasil_preco_diesel_s10
    print('Já contém o mês atual em questão, valores atualizados!')
else:
    # Adicionar uma nova linha com os valores adequados
    nova_linha_brasil_diesel = {'MÊS': data_mes_formatada, 'PRODUTO': 'OLEO DIESEL', 'PREÇO MÉDIO REVENDA': media_brasil_preco_diesel}
    nova_linha_brasil_diesel_s10 = {'MÊS': data_mes_formatada, 'PRODUTO': 'OLEO DIESEL S10', 'PREÇO MÉDIO REVENDA': media_brasil_preco_diesel_s10}
    df_brasil_ano = df_brasil_ano.append(nova_linha_brasil_diesel, ignore_index=True)
    df_brasil_ano = df_brasil_ano.append(nova_linha_brasil_diesel_s10, ignore_index=True)
    print('Não possui o mês em questão, adicionando novas linhas com os valores referente ao mês em questão!')
# Exibir as últimas 6 linhas de df_sp_ano
ultimas_linhas_brasil_ano = df_brasil_ano.tail(6)
print(ultimas_linhas_brasil_ano)

Já contém o mês atual em questão, valores atualizados!
0        MÊS          PRODUTO PREÇO MÉDIO REVENDA 11/03/2024
117  jan.-24      OLEO DIESEL                5.85       None
118  jan.-24  OLEO DIESEL S10                5.94       None
119  fev.-24      OLEO DIESEL              5.8975       None
120  fev.-24  OLEO DIESEL S10                5.97       None
121  mar.-24      OLEO DIESEL                5.87       None
122  mar.-24  OLEO DIESEL S10               5.935       None


In [32]:
# Limpar a aba "New Gas Database - SP ANO" antes de colar os novos dados
aba_brasil_ano.delete_rows(1, aba_brasil_ano.max_row)

# Converter o DataFrame para linhas
rows = dataframe_to_rows(df_brasil_ano, index=False, header=True)

# Colar os dados na aba "New Gas Database - SP ANO"
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
        aba_brasil_ano.cell(row=r_idx, column=c_idx, value=value)
        
# Obter a letra da coluna 'D'
column_letter = get_column_letter(4)  # 'D' é a quarta letra do alfabeto
# Adicionar a data atual na célula D1
aba_brasil_ano[column_letter + '1'] = date.today().strftime('%d/%m/%Y')

In [33]:
# Salvar as alterações na planilha base
workbook_base.save(caminho_planilha_base)
print(f'Dados colados com sucesso na aba "New Gas Database" da planilha base.')

Dados colados com sucesso na aba "New Gas Database" da planilha base.


# ANÁLISES

In [34]:
# Carregar a aba 'BRASIL' como um DataFrame
df_brasil = pd.DataFrame(workbook_base['BRASIL'].values)

# Definir a primeira linha como cabeçalho
df_brasil.columns = df_brasil.iloc[0]

# Remover a primeira linha (cabeçalho)
df_brasil = df_brasil.iloc[1:]

# Converter a coluna 'DATA INICIAL' para o tipo datetime
df_brasil['DATA INICIAL'] = pd.to_datetime(df_brasil['DATA INICIAL'], format='%d/%m/%Y')

# Filtrar os dados pela data atual
df_brasil_atual = df_brasil[df_brasil['DATA INICIAL'] == data_atual]

# Selecionar apenas as colunas desejadas
produtos_combustiveis = ['OLEO DIESEL', 'OLEO DIESEL S10', 'GASOLINA COMUM', 'GASOLINA ADITIVADA', 'ETANOL HIDRATADO', 'GNV']

# Ordenar os produtos na ordem desejada
produtos_ordem = ['OLEO DIESEL', 'OLEO DIESEL S10', 'GASOLINA COMUM', 'GASOLINA ADITIVADA', 'ETANOL HIDRATADO', 'GNV']
precos_medios_revenda_brasil = df_brasil_atual[df_brasil_atual['PRODUTO'].isin(produtos_ordem)]

# Formatar o print desejado
print("No Brasil o PREÇO MÉDIO REVENDA dos produtos na data ({} {}) foram:\n".format(data_atual.strftime('%d-%m-%Y'), data_atual.strftime('%H:%M:%S')))
for produto in produtos_ordem:
    preco_medio = precos_medios_revenda_brasil.loc[precos_medios_revenda_brasil['PRODUTO'] == produto, 'PREÇO MÉDIO REVENDA'].iloc[0]
    print(f"{produto:<20} {preco_medio}")




No Brasil o PREÇO MÉDIO REVENDA dos produtos na data (10-03-2024 00:00:00) foram:

OLEO DIESEL          5.86
OLEO DIESEL S10      5.92
GASOLINA COMUM       5.73
GASOLINA ADITIVADA   5.93
ETANOL HIDRATADO     3.58
GNV                  4.53


In [35]:
# Carregar a aba 'CAPITAIS' como um DataFrame
df_capitais_att = pd.DataFrame(workbook_base['CAPITAIS'].values)

# Definir a primeira linha como cabeçalho
df_capitais_att.columns = df_capitais_att.iloc[0]

# Remover a primeira linha (cabeçalho)
df_capitais_att = df_capitais_att.iloc[1:]

# Converter a coluna 'DATA INICIAL' para o tipo datetime
df_capitais_att['DATA INICIAL'] = pd.to_datetime(df_capitais_att['DATA INICIAL'], format='%d/%m/%Y')

# Filtrar os dados pela data atual
df_capitais_att = df_capitais_att[df_capitais_att['DATA INICIAL'] == data_atual]

# Filtrar os dados pelo estado SÃO PAULO
df_capitais_att = df_capitais_att[df_capitais_att['ESTADO'].isin(['SAO PAULO'])]

# Selecionar apenas as colunas desejadas
produtos_combustiveis = ['OLEO DIESEL', 'OLEO DIESEL S10', 'GASOLINA COMUM', 'GASOLINA ADITIVADA', 'ETANOL HIDRATADO', 'GNV']

# Ordenar os produtos na ordem desejada
produtos_ordem = ['OLEO DIESEL', 'OLEO DIESEL S10', 'GASOLINA COMUM', 'GASOLINA ADITIVADA', 'ETANOL HIDRATADO', 'GNV']
precos_medios_revenda_sp = df_capitais_att[df_capitais_att['PRODUTO'].isin(produtos_ordem)]

# Formatar o print desejado
print("Em SÃO PAULO o PREÇO MÉDIO REVENDA dos produtos na data ({} {}) foram:\n".format(data_atual.strftime('%d-%m-%Y'), data_atual.strftime('%H:%M:%S')))
for produto in produtos_ordem:
    preco_medio = precos_medios_revenda_sp.loc[precos_medios_revenda_sp['PRODUTO'] == produto, 'PREÇO MÉDIO REVENDA'].iloc[0]
    print(f"{produto:<20} {preco_medio}")

Em SÃO PAULO o PREÇO MÉDIO REVENDA dos produtos na data (10-03-2024 00:00:00) foram:

OLEO DIESEL          5.82
OLEO DIESEL S10      6.07
GASOLINA COMUM       5.67
GASOLINA ADITIVADA   6.02
ETANOL HIDRATADO     3.48
GNV                  4.3


# COMPARATIVO VALORES DAS CAPITAIS SEMANA ATUAL COM SEMANA ANTERIOR

In [36]:
# Converter data_anterior para um objeto Timestamp do pandas
data_anterior_timestamp = pd.to_datetime(data_anterior)

# Formatar as datas utilizando o método strftime
data_atual_formatada_comparativo = data_atual.strftime('%Y-%m-%d %H:%M:%S')
data_anterior_formatada_comparativo = data_anterior_timestamp.strftime('%Y-%m-%d %H:%M:%S')

semana_atual = data_atual_formatada_comparativo
semana_anterior = data_anterior_formatada_comparativo
print(semana_atual)
print(semana_anterior)

2024-03-10 00:00:00
2024-03-03 00:00:00


In [37]:
# Caminho do arquivo base
caminho_base = r'C:\Users\ricardo\Analise combustiveis\analise diesel\base_diesel_desde__31_07_2022.xlsx'

# Carregar o arquivo base
workbook_base = load_workbook(filename=caminho_base)

In [38]:
# Selecionar a aba "CAPITAIS" no arquivo base
aba_capitais_base_comparativo = workbook_base['CAPITAIS']

# Converter a aba "CAPITAIS" em um DataFrame
df_capitais_base_comparativo = pd.DataFrame(aba_capitais_base_comparativo.values, columns=[cell.value for cell in aba_capitais_base_comparativo[1]])

# Converter a coluna 'DATA INICIAL' para o formato de data, pulando a primeira linha
df_capitais_base_comparativo['DATA INICIAL'] = pd.to_datetime(df_capitais_base_comparativo['DATA INICIAL'], errors='coerce')

# Formatar as datas na coluna 'DATA INICIAL' para o formato 'dd/mm/aaaa'
df_capitais_base_comparativo['DATA INICIAL'] = df_capitais_base_comparativo['DATA INICIAL'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Lista de capitais
capitais = [
    'ARACAJU', 'BELEM', 'BELO HORIZONTE', 'BOA VISTA', 'BRASILIA', 'CAMPO GRANDE',
    'CUIABA', 'CURITIBA', 'FLORIANOPOLIS', 'FORTALEZA', 'GOIANIA', 'JOAO PESSOA',
    'MACAPA', 'MACEIO', 'MANAUS', 'NATAL', 'PALMAS', 'PORTO ALEGRE', 'PORTO VELHO',
    'RECIFE', 'RIO BRANCO', 'RIO DE JANEIRO', 'SALVADOR', 'SAO LUIS', 'SAO PAULO', 'TERESINA', 'VITORIA'
]

# Filtrar os dados da aba 'CAPITAIS' para incluir apenas as capitais da lista e a data atual
df_capitais_filtrado_base = df_capitais_base_comparativo[df_capitais_base_comparativo['DATA INICIAL'] == semana_atual]
df_capitais_filtrado_base = df_capitais_filtrado_base[df_capitais_filtrado_base['MUNICÍPIO'].isin(capitais)]

# Pivotar a tabela para ter os produtos como colunas
df_pivot_base_atual = df_capitais_filtrado_base.pivot(index='MUNICÍPIO', columns='PRODUTO', values='PREÇO MÉDIO REVENDA')

# Selecionar apenas as linhas de interesse (óleo diesel comum e óleo diesel S10)
df_resultado_base_atual = df_pivot_base_atual[['OLEO DIESEL', 'OLEO DIESEL S10']]

# Remover as linhas com valores NaN
df_resultado_limpo_atual = df_resultado_base_atual.dropna()

# Resetar o índice para transformar "MUNICÍPIO" em uma coluna
df_resultado_limpo_atual.reset_index(inplace=True)

# Renomear a coluna "MUNICÍPIO" para "CAPITAIS"
df_resultado_limpo_atual.rename(columns={'MUNICÍPIO': 'CAPITAIS'}, inplace=True)

# Selecionar apenas as colunas desejadas
df_resultado_limpo_atual = df_resultado_limpo_atual[['CAPITAIS', 'OLEO DIESEL S10', 'OLEO DIESEL']]

# Exibir o resultado
print(f"Resultado da semana atual da data: {semana_atual}")
print(df_resultado_limpo_atual)

Resultado da semana atual da data: 2024-03-10 00:00:00
PRODUTO        CAPITAIS OLEO DIESEL S10 OLEO DIESEL
0               ARACAJU            5.93        6.24
1        BELO HORIZONTE            5.83        5.66
2             BOA VISTA            6.68        6.63
3              BRASILIA            5.92        5.83
4          CAMPO GRANDE            5.96        5.89
5                CUIABA            5.91         5.9
6              CURITIBA             5.9        5.62
7               GOIANIA            5.94        5.82
8           JOAO PESSOA            5.74        5.64
9                MACAPA            6.54        6.59
10               MANAUS            6.29        6.29
11                NATAL            5.97        5.96
12               PALMAS            5.64        5.69
13         PORTO ALEGRE            5.89        5.86
14          PORTO VELHO             6.4        6.44
15           RIO BRANCO            7.15         7.1
16       RIO DE JANEIRO            6.05        5.89
17       

C:\Users\ricardo\AppData\Local\Temp\ipykernel_13608\4172552903.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado_limpo_atual.rename(columns={'MUNICÍPIO': 'CAPITAIS'}, inplace=True)


In [39]:
# Selecionar a aba "CAPITAIS" no arquivo base
aba_capitais_base_comparativo = workbook_base['CAPITAIS']

# Converter a aba "CAPITAIS" em um DataFrame
df_capitais_base_comparativo = pd.DataFrame(aba_capitais_base_comparativo.values, columns=[cell.value for cell in aba_capitais_base_comparativo[1]])

# Converter a coluna 'DATA INICIAL' para o formato de data, pulando a primeira linha
df_capitais_base_comparativo['DATA INICIAL'] = pd.to_datetime(df_capitais_base_comparativo['DATA INICIAL'], errors='coerce')

# Formatar as datas na coluna 'DATA INICIAL' para o formato 'dd/mm/aaaa'
df_capitais_base_comparativo['DATA INICIAL'] = df_capitais_base_comparativo['DATA INICIAL'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Lista de capitais
capitais = [
    'ARACAJU', 'BELEM', 'BELO HORIZONTE', 'BOA VISTA', 'BRASILIA', 'CAMPO GRANDE',
    'CUIABA', 'CURITIBA', 'FLORIANOPOLIS', 'FORTALEZA', 'GOIANIA', 'JOAO PESSOA',
    'MACAPA', 'MACEIO', 'MANAUS', 'NATAL', 'PALMAS', 'PORTO ALEGRE', 'PORTO VELHO',
    'RECIFE', 'RIO BRANCO', 'RIO DE JANEIRO', 'SALVADOR', 'SAO LUIS', 'SAO PAULO', 'TERESINA', 'VITORIA'
]

# Filtrar os dados da aba 'CAPITAIS' para incluir apenas as capitais da lista e a data anterior
df_capitais_filtrado_base = df_capitais_base_comparativo[df_capitais_base_comparativo['DATA INICIAL'] == semana_anterior]
df_capitais_filtrado_base = df_capitais_filtrado_base[df_capitais_filtrado_base['MUNICÍPIO'].isin(capitais)]

# Pivotar a tabela para ter os produtos como colunas
df_pivot_base_anterior = df_capitais_filtrado_base.pivot(index='MUNICÍPIO', columns='PRODUTO', values='PREÇO MÉDIO REVENDA')

# Selecionar apenas as linhas de interesse (óleo diesel comum e óleo diesel S10)
df_resultado_base_anterior = df_pivot_base_anterior[['OLEO DIESEL', 'OLEO DIESEL S10']]

# Remover as linhas com valores NaN
df_resultado_limpo_anterior = df_resultado_base_anterior.dropna()

# Resetar o índice para transformar "MUNICÍPIO" em uma coluna
df_resultado_limpo_anterior.reset_index(inplace=True)

# Renomear a coluna "MUNICÍPIO" para "CAPITAIS"
df_resultado_limpo_anterior.rename(columns={'MUNICÍPIO': 'CAPITAIS'}, inplace=True)

# Selecionar apenas as colunas desejadas
df_resultado_limpo_anterior = df_resultado_limpo_anterior[['CAPITAIS', 'OLEO DIESEL S10', 'OLEO DIESEL']]

# Exibir o resultado
print(f"Resultado da semana anterior da data: {semana_anterior}")
print(df_resultado_limpo_anterior)

Resultado da semana anterior da data: 2024-03-03 00:00:00
PRODUTO        CAPITAIS OLEO DIESEL S10 OLEO DIESEL
0               ARACAJU            5.92        6.24
1        BELO HORIZONTE            5.81        5.68
2             BOA VISTA            6.68        6.64
3              BRASILIA            5.87        5.82
4          CAMPO GRANDE            5.96         5.8
5                CUIABA            5.94        5.92
6              CURITIBA            5.96        5.69
7               GOIANIA            5.94        5.83
8           JOAO PESSOA            5.76        5.58
9                MACAPA            6.51        6.64
10               MANAUS            6.29        6.29
11                NATAL            6.07        5.79
12               PALMAS            5.59        5.62
13         PORTO ALEGRE            5.92        5.96
14          PORTO VELHO            6.38        6.39
15           RIO BRANCO            7.18        7.11
16       RIO DE JANEIRO            6.06        5.87
17    

C:\Users\ricardo\AppData\Local\Temp\ipykernel_13608\3078648009.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resultado_limpo_anterior.rename(columns={'MUNICÍPIO': 'CAPITAIS'}, inplace=True)


In [40]:
# Calcular a diferença entre os DataFrames df_resultado_limpo_atual e df_resultado_limpo_anterior
df_diferenca = df_resultado_limpo_atual.set_index('CAPITAIS') - df_resultado_limpo_anterior.set_index('CAPITAIS')

# Calcular a variação percentual
df_variacao_percentual = ((df_resultado_limpo_atual.set_index('CAPITAIS') - df_resultado_limpo_anterior.set_index('CAPITAIS')) / df_resultado_limpo_anterior.set_index('CAPITAIS')) * 100

# Concatenar os resultados em um novo DataFrame
df_comparativo = pd.concat([df_diferenca, df_variacao_percentual], axis=1, keys=['Variação Absoluta', 'Variação Percentual'])
df_comparativo = df_comparativo.dropna()

# Exibir o DataFrame final
print(df_comparativo)

               Variação Absoluta             Variação Percentual            
PRODUTO          OLEO DIESEL S10 OLEO DIESEL     OLEO DIESEL S10 OLEO DIESEL
CAPITAIS                                                                    
ARACAJU                     0.01         0.0            0.168919         0.0
BELO HORIZONTE              0.02       -0.02            0.344234   -0.352113
BOA VISTA                    0.0       -0.01                 0.0   -0.150602
BRASILIA                    0.05        0.01            0.851789    0.171821
CAMPO GRANDE                 0.0        0.09                 0.0    1.551724
CUIABA                     -0.03       -0.02           -0.505051   -0.337838
CURITIBA                   -0.06       -0.07           -1.006711   -1.230228
GOIANIA                      0.0       -0.01                 0.0   -0.171527
JOAO PESSOA                -0.02        0.06           -0.347222    1.075269
MACAPA                      0.03       -0.05            0.460829   -0.753012

In [41]:
# Filtrar as capitais que tiveram uma variação absoluta maior que 2% (positiva ou negativa)
df_variacao_maior_que_2 = df_comparativo[(df_comparativo['Variação Percentual']['OLEO DIESEL S10'].abs() > 2) | (df_comparativo['Variação Percentual']['OLEO DIESEL'].abs() > 2)]

# Ordenar as capitais filtradas em ordem alfabética
df_variacao_maior_que_2_ordenado = df_variacao_maior_que_2.sort_index()

# Selecionar até as 5 primeiras capitais após a ordenação
df_top_5_capitais = df_variacao_maior_que_2_ordenado.head(5)

# Incluir São Paulo no DataFrame
if 'SAO PAULO' not in df_top_5_capitais.index:
    df_top_5_capitais.loc['SAO PAULO'] = df_comparativo.loc['SAO PAULO']

# Exibir o resultado
print(df_top_5_capitais)


          Variação Absoluta             Variação Percentual            
PRODUTO     OLEO DIESEL S10 OLEO DIESEL     OLEO DIESEL S10 OLEO DIESEL
CAPITAIS                                                               
NATAL                  -0.1        0.17           -1.647446    2.936097
SAO LUIS              -0.02        0.19           -0.364964    3.584906
SAO PAULO               0.0        0.02                 0.0    0.344828


C:\Users\ricardo\AppData\Local\Temp\ipykernel_13608\236894480.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_5_capitais.loc['SAO PAULO'] = df_comparativo.loc['SAO PAULO']


In [44]:
# Converter as datas para o formato desejado (%d-%m-%Y)
semana_anterior_formatada = datetime.strptime(semana_anterior, "%Y-%m-%d %H:%M:%S")
semana_atual_formatada = datetime.strptime(semana_atual, "%Y-%m-%d %H:%M:%S")

# Solicitar a ultima data da semana_atual
nova_data = input("Por favor, informe a data da ultima data da semana_atual no formato (dia/mês/ano): ")
nova_data_formatada = datetime.strptime(nova_data, "%d/%m/%Y")

# Calcular a data da semana anterior à próxima semana
semana_anterior_proxima = nova_data_formatada - timedelta(days=7)

# Construir o texto
texto = f"""
O Painel do Diesel foi atualizado com os dados da semana {semana_atual_formatada.strftime("%d/%m/%Y")} a {nova_data_formatada.strftime("%d/%m/%Y")}.

- Conforme análise comparativa da semana {semana_anterior_formatada.strftime("%d/%m/%Y")} a {semana_anterior_proxima.strftime("%d/%m/%Y")} com a semana {semana_atual_formatada.strftime("%d/%m/%Y")} a {nova_data_formatada.strftime("%d/%m/%Y")}, foram identificados alguns indicadores relevantes, sendo eles:
"""

# Exibir o texto
print(texto)

# Obter os preços atuais de diesel comum e diesel S10 para cada capital
precos_atuais = df_resultado_limpo_atual.set_index('CAPITAIS')[['OLEO DIESEL', 'OLEO DIESEL S10']]

# Filtrar os valores de variação maiores que 2% ou menores que -2%
filtro_variacao_maior_que_2 = df_top_5_capitais[('Variação Percentual', 'OLEO DIESEL')].abs() > 2
filtro_variacao_menor_que_menos2 = df_top_5_capitais[('Variação Percentual', 'OLEO DIESEL')].abs() < -2
filtro_variacao_diesel_s10_maior_que_2 = df_top_5_capitais[('Variação Percentual', 'OLEO DIESEL S10')].abs() > 2
filtro_variacao_diesel_s10_menor_que_menos2 = df_top_5_capitais[('Variação Percentual', 'OLEO DIESEL S10')].abs() < -2

# Aplicar os filtros
df_top_5_capitais_filtrado = df_top_5_capitais[(filtro_variacao_maior_que_2 | filtro_variacao_menor_que_menos2 | filtro_variacao_diesel_s10_maior_que_2 | filtro_variacao_diesel_s10_menor_que_menos2)]

# Inicializar uma lista para armazenar as informações de cada capital
texto_capitais = []

# Percorrer as capitais do DataFrame filtrado
for capital, variacoes in df_top_5_capitais_filtrado.iterrows():
    # Inicializar uma lista para armazenar as informações de variação para a capital atual
    texto_variacoes = []
    
    # Verificar se há variação de preço do diesel comum maior que 2% ou menor que -2%
    if abs(variacoes[('Variação Percentual', 'OLEO DIESEL')]) > 2:
        # Extrair informações da variação do diesel comum
        var_diesel_comum_percentual = variacoes[('Variação Percentual', 'OLEO DIESEL')]
        preco_atual_diesel_comum = precos_atuais.loc[capital, 'OLEO DIESEL']
        # Gerar o texto correspondente
        if var_diesel_comum_percentual > 0:
            texto_diesel_comum = f"Aumentou o valor do diesel comum em {abs(var_diesel_comum_percentual):.2f}% (Valor atual: R$ {preco_atual_diesel_comum:.2f})."
        elif var_diesel_comum_percentual < 0:
            texto_diesel_comum = f"Diminuiu o valor do diesel comum em -{abs(var_diesel_comum_percentual):.2f}% (Valor atual: R$ {preco_atual_diesel_comum:.2f})."
        texto_variacoes.append(texto_diesel_comum)
        
    # Verificar se há variação de preço do diesel S10 maior que 2% ou menor que -2%
    if abs(variacoes[('Variação Percentual', 'OLEO DIESEL S10')]) > 2:
        # Extrair informações da variação do diesel S10
        var_diesel_s10_percentual = variacoes[('Variação Percentual', 'OLEO DIESEL S10')]
        preco_atual_diesel_s10 = precos_atuais.loc[capital, 'OLEO DIESEL S10']
        # Gerar o texto correspondente
        if var_diesel_s10_percentual > 0:
            texto_diesel_s10 = f"Aumentou o valor do diesel S10 em {abs(var_diesel_s10_percentual):.2f}% (Valor atual: R$ {preco_atual_diesel_s10:.2f})."
        elif var_diesel_s10_percentual < 0:
            texto_diesel_s10 = f"Diminuiu o valor do diesel S10 em -{abs(var_diesel_s10_percentual):.2f}% (Valor atual: R$ {preco_atual_diesel_s10:.2f})."
        texto_variacoes.append(texto_diesel_s10)
        
    # Adicionar o nome da capital ao texto
    texto_capital = capital + " – " + " E ".join(texto_variacoes)
    texto_capitais.append(texto_capital)

# Concatenar os textos das capitais
texto_parte2_corrigido = "\n".join(texto_capitais)

# Adicionar informações de São Paulo mesmo se não houver variação maior que 2% ou menor que -2%
var_diesel_comum_sp = df_resultado_limpo_atual.loc[df_resultado_limpo_atual['CAPITAIS'] == 'SAO PAULO', 'OLEO DIESEL'].values[0]
var_diesel_s10_sp = df_resultado_limpo_atual.loc[df_resultado_limpo_atual['CAPITAIS'] == 'SAO PAULO', 'OLEO DIESEL S10'].values[0]

texto_diesel_s10_sp = ""
texto_diesel_comum_sp = ""

# Inicializar uma lista para armazenar as informações de variação para São Paulo
texto_variacoes_sp = []

# Verificar se houve variação de preço do diesel comum em São Paulo
if abs(var_diesel_comum_sp) > 2:
    # Gerar o texto correspondente
    variacao_percentual_diesel_comum_sp = df_top_5_capitais.loc[('SAO PAULO', 'Variação Percentual')]
    variacao_percentual_diesel_comum_sp = round(variacao_percentual_diesel_comum_sp.loc["OLEO DIESEL"], 2)
    if variacao_percentual_diesel_comum_sp > 0:
        texto_diesel_comum_sp = f"Aumentou o valor do diesel comum em {abs(variacao_percentual_diesel_comum_sp):.2f}% (Valor atual: R$ {var_diesel_comum_sp:.2f})."
    elif variacao_percentual_diesel_comum_sp < 0:
        texto_diesel_comum_sp = f"Diminuiu o valor do diesel comum em -{abs(variacao_percentual_diesel_comum_sp):.2f}% (Valor atual: R$ {var_diesel_comum_sp:.2f})."        
    texto_variacoes_sp.append(texto_diesel_comum_sp)

# Verificar se houve variação de preço do diesel S10 em São Paulo
if abs(var_diesel_s10_sp) > 2:
    # Gerar o texto correspondente
    variacao_percentual_diesel_s10_sp = df_top_5_capitais.loc[('SAO PAULO', 'Variação Percentual')]
    variacao_percentual_diesel_s10_sp = round(variacao_percentual_diesel_s10_sp.loc["OLEO DIESEL S10"], 2)
    if variacao_percentual_diesel_s10_sp > 0:
        texto_diesel_s10_sp = f"Aumentou o valor do diesel S10 em {abs(variacao_percentual_diesel_s10_sp):.2f}% (Valor atual: R$ {var_diesel_s10_sp:.2f})."
    elif variacao_percentual_diesel_s10_sp < 0:
        texto_diesel_s10_sp = f"Diminuiu o valor do diesel S10 em -{abs(variacao_percentual_diesel_s10_sp):.2f}% (Valor atual: R$ {var_diesel_s10_sp:.2f})."        
    texto_variacoes_sp.append(texto_diesel_s10_sp)

# Adicionar São Paulo ao texto das capitais
texto_sp = "SAO PAULO – " + " E ".join(texto_variacoes_sp)

# Adicionar São Paulo ao final da lista de capitais
texto_capitais.append(texto_sp)

# Concatenar os textos das capitais, incluindo uma linha em branco após cada capital
texto_parte2_final = "\n\n".join(texto_capitais)

# Exibir o texto final
print(texto_parte2_final)

# Calcular o preço médio para cada capital
df_resultado_limpo_atual['Preço Médio'] = (df_resultado_limpo_atual['OLEO DIESEL S10'] + df_resultado_limpo_atual['OLEO DIESEL']) / 2

# Converter a coluna 'Preço Médio' para tipo numérico
df_resultado_limpo_atual['Preço Médio'] = pd.to_numeric(df_resultado_limpo_atual['Preço Médio'])

# Encontrar o índice do preço médio mais alto e mais baixo
indice_diesel_mais_caro = df_resultado_limpo_atual['Preço Médio'].idxmax()
indice_diesel_mais_barato = df_resultado_limpo_atual['Preço Médio'].idxmin()

# Extrair as informações das capitais com os preços mais caro e mais barato
capital_diesel_mais_caro = df_resultado_limpo_atual.loc[indice_diesel_mais_caro, 'CAPITAIS']
capital_diesel_mais_barato = df_resultado_limpo_atual.loc[indice_diesel_mais_barato, 'CAPITAIS']

# Retornar os resultados formatados
print("\n*A capital com diesel MAIS CARO é", capital_diesel_mais_caro, "com os seguintes valores:")
print("Diesel S10:", "R$", df_resultado_limpo_atual.loc[indice_diesel_mais_caro, 'OLEO DIESEL S10'])
print("Diesel comum:", "R$", df_resultado_limpo_atual.loc[indice_diesel_mais_caro, 'OLEO DIESEL'])
print("\n*A capital com diesel MAIS BARATO é", capital_diesel_mais_barato, "com os seguintes valores:")
print("Diesel S10:", "R$", df_resultado_limpo_atual.loc[indice_diesel_mais_barato, 'OLEO DIESEL S10'])
print("Diesel comum:", "R$", df_resultado_limpo_atual.loc[indice_diesel_mais_barato, 'OLEO DIESEL'])

Por favor, informe a data da ultima data da semana_atual no formato (dia/mês/ano): 16/03/2024

O Painel do Diesel foi atualizado com os dados da semana 10/03/2024 a 16/03/2024.

- Conforme análise comparativa da semana 03/03/2024 a 09/03/2024 com a semana 10/03/2024 a 16/03/2024, foram identificados alguns indicadores relevantes, sendo eles:

NATAL – Aumentou o valor do diesel comum em 2.94% (Valor atual: R$ 5.96).

SAO LUIS – Aumentou o valor do diesel comum em 3.58% (Valor atual: R$ 5.49).

SAO PAULO – Aumentou o valor do diesel comum em 0.34% (Valor atual: R$ 5.82). E 

*A capital com diesel MAIS CARO é RIO BRANCO com os seguintes valores:
Diesel S10: R$ 7.15
Diesel comum: R$ 7.1

*A capital com diesel MAIS BARATO é SAO LUIS com os seguintes valores:
Diesel S10: R$ 5.46
Diesel comum: R$ 5.49


In [45]:
# Definir os valores de variação para OLEO DIESEL e OLEO DIESEL S10
var_diesel = -0.86
var_diesel_s10 = -0.28

# Função para classificar a variação percentual
def classificar_variacao(variacao):
    if variacao < -1:
        return "diminuição considerável"
    elif -0.7 <= variacao < -0.99:
        return "diminuição"
    elif -0.49 <= variacao < -0.7:
        return "leve diminuição"
    elif 0.49 <= variacao < 0.7:
        return "leve aumento"
    elif 0.7 <= variacao < 0.99:
        return "aumento"
    elif variacao > 1:
        return "aumento considerável"
    else:
        return "pouca variação"  # Atualização aqui: classificar como "pouca variação"

# Verificar se os valores estão entre -0.49% e 0.49%
if -0.49 <= var_diesel <= 0.49 and -0.49 <= var_diesel_s10 <= 0.49:
    print("\nVeja o texto da última semana:")
    print("'Esta semana, observou-se um cenário de leve aumento nos preços do diesel comum, enquanto o diesel S10 permaneceu em geral estável. Essa pequena variação reflete um período de relativa estabilidade nos preços dos combustíveis, proporcionando aos consumidores um ambiente mais previsível. No entanto, é essencial estar atento às possíveis influências externas que podem afetar o mercado no futuro.'")
    print("\nPara esta semana, o cenário tanto para Diesel comum quanto para Diesel S10 se encontra com pouca variação. Segue o mesmo modelo do texto, somente um parágrafo.")

# Verificar se os valores são maiores que 0.49% ou menores que -0.49%
elif var_diesel > 0.49 or var_diesel < -0.49 or var_diesel_s10 > 0.49 or var_diesel_s10 < -0.49:
    print("\nVeja o texto da última semana:")
    aumento_diminuicao_diesel = classificar_variacao(var_diesel)
    aumento_diminuicao_diesel_s10 = classificar_variacao(var_diesel_s10)
    print(f"'Esta semana, observou-se um cenário de leve aumento nos preços do diesel comum, enquanto o diesel S10 permaneceu em geral estável. Essa pequena variação reflete um período de relativa estabilidade nos preços dos combustíveis, proporcionando aos consumidores um ambiente mais previsível. No entanto, é essencial estar atento às possíveis influências externas que podem afetar o mercado no futuro.'")
    print(f"\nPara esta semana, o cenário para Diesel comum foi de {aumento_diminuicao_diesel} no geral, e para Diesel S10 foi de {aumento_diminuicao_diesel_s10} no geral. Segue o mesmo modelo do texto, somente um parágrafo.")



Veja o texto da última semana:
'Esta semana, observou-se um cenário de leve aumento nos preços do diesel comum, enquanto o diesel S10 permaneceu em geral estável. Essa pequena variação reflete um período de relativa estabilidade nos preços dos combustíveis, proporcionando aos consumidores um ambiente mais previsível. No entanto, é essencial estar atento às possíveis influências externas que podem afetar o mercado no futuro.'

Para esta semana, o cenário para Diesel comum foi de pouca variação no geral, e para Diesel S10 foi de pouca variação no geral. Segue o mesmo modelo do texto, somente um parágrafo.


In [46]:
print(var_diesel)
print(var_diesel_s10)

-0.86
-0.28
